In [ ]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import faiss
from rag.embed_query import embed_query
##mas que nada lo cree para ver los metadatos
load_dotenv()

INDEX_PATH = os.path.join("data", "processed", "index.faiss")
METADATA_PATH = os.path.join("data", "processed", "chunks.parquet")

print("✅ Paths cargados correctamente")
print(f"Index FAISS: {INDEX_PATH}")
print(f"Metadata: {METADATA_PATH}")

✅ Paths cargados correctamente
Index FAISS: data\processed\index.faiss
Metadata: data\processed\chunks.parquet


In [10]:
index = faiss.read_index(INDEX_PATH)
print("✅ FAISS index cargado")

# Cargar metadata
metadata = pd.read_parquet(METADATA_PATH)
print("✅ Metadata cargada")
print(f"Número de chunks: {len(metadata)}")
metadata.head(5)


✅ FAISS index cargado
✅ Metadata cargada
Número de chunks: 97


,doc_id,chunk_id,text,metadata
0,doc_003,data_0_0,## Chunk 1 **Calendario Académico 2025** ### E...,"{'chunk_id': 'data_0_0', 'doc_id': 'doc_003', ..."
1,doc_003,data_0_1,Jueves 01 de mayo Feriado Nacional – Día del T...,"{'chunk_id': 'data_0_1', 'doc_id': 'doc_003', ..."
2,doc_003,data_0_2,4º bimestre. ([ufro.cl](https://www.ufro.cl/ca...,"{'chunk_id': 'data_0_2', 'doc_id': 'doc_003', ..."
3,doc_001,REGLAMENTO-DE-CONVIVENCIA-UNIVERSITARIA_1_0,/gid00051/gid00038/gid00040/gid00045/gid00034/...,{'chunk_id': 'REGLAMENTO-DE-CONVIVENCIA-UNIVER...
4,doc_001,REGLAMENTO-DE-CONVIVENCIA-UNIVERSITARIA_2_0,DIRECCIÓN DE EQUIDAD DE GÉNERO UNIVERSIDAD DE ...,{'chunk_id': 'REGLAMENTO-DE-CONVIVENCIA-UNIVER...


In [11]:
print("Columnas en metadata:", metadata.columns.tolist())

# Ver si hay columna 'metadata'
if 'metadata' in metadata.columns:
    print("✅ La columna 'metadata' existe. Algunos ejemplos:")
    print(metadata['metadata'].head(5))
else:
    print("⚠ No existe columna 'metadata', se usarán diccionarios vacíos en su lugar.")

Columnas en metadata: ['doc_id', 'chunk_id', 'text', 'metadata']
✅ La columna 'metadata' existe. Algunos ejemplos:
0    {'chunk_id': 'data_0_0', 'doc_id': 'doc_003', ...
1    {'chunk_id': 'data_0_1', 'doc_id': 'doc_003', ...
2    {'chunk_id': 'data_0_2', 'doc_id': 'doc_003', ...
3    {'chunk_id': 'REGLAMENTO-DE-CONVIVENCIA-UNIVER...
4    {'chunk_id': 'REGLAMENTO-DE-CONVIVENCIA-UNIVER...
Name: metadata, dtype: object


In [12]:

query = "Artículo 3 del reglamento de convivencia"
embedding = embed_query(query)

print(f"Embedding generado. Shape: {embedding.shape}")
print("Primeros 10 valores:", embedding[0, :10])


Embedding generado. Shape: (1, 384)
Primeros 10 valores: [-0.00598394  0.03019341 -0.04273844 -0.03170299 -0.13058773  0.00878726
  0.05249117 -0.01990112  0.03660937  0.03762356]


In [13]:
k = 3
if len(embedding.shape) == 1:
    embedding = embedding.reshape(1, -1)

scores, idxs = index.search(embedding.astype("float32"), k)
print(f"Chunks más cercanos (k={k}):")
for score, i in zip(scores[0], idxs[0]):
    if i == -1:
        continue
    doc = metadata.iloc[i]
    content_preview = doc['text'][:150].replace("\n", " ")
    print(f"- Score: {score:.3f}, Chunk {i}: {content_preview}...")
    if 'metadata' in doc:
        print(f"  Metadata: {doc['metadata']}")
    else:
        print("  Metadata: {}")

Chunks más cercanos (k=3):
- Score: 0.656, Chunk 49: 04/ RREP TÍTULO I: DISPOSICIONES GENERALES ARTÍCULO 1º: El Reglamento de Régimen de Estudios de Pregrado, tiene por objeto establecer las disposicione...
  Metadata: {'chunk_id': 'ReglamentodeRegimendeEstudios__2023__5_0', 'doc_id': 'doc_002', 'filename': 'data/raw/ReglamentodeRegimendeEstudios__2023_.pdf', 'page': 5.0, 'source_type': 'pdf', 'title': 'ReglamentodeRegimendeEstudios  2023 ', 'url': 'https://www.ufro.cl', 'vigencia': None}
- Score: 0.580, Chunk 56: REGLAMENTO DE RÉGIMEN DE ESTUDIO DE PREGRADO 11 / RREP especializada, en concordancia con lo expresado en el perﬁl deﬁnido para cada carrera. El máxim...
  Metadata: {'chunk_id': 'ReglamentodeRegimendeEstudios__2023__12_0', 'doc_id': 'doc_002', 'filename': 'data/raw/ReglamentodeRegimendeEstudios__2023_.pdf', 'page': 12.0, 'source_type': 'pdf', 'title': 'ReglamentodeRegimendeEstudios  2023 ', 'url': 'https://www.ufro.cl', 'vigencia': None}
- Score: 0.564, Chunk 82: REGLAMENTO 

In [14]:
print("Ejemplos aleatorios de 5 chunks:")
for _, doc in metadata.sample(5).iterrows():
    print("--------------------------------------------------------")
    print("Chunk:", doc['text'][:200].replace("\n", " "), "...")
    if 'metadata' in doc:
        print("Metadata:", doc['metadata'])
    else:
        print("Metadata: {}")


Ejemplos aleatorios de 5 chunks:
--------------------------------------------------------
Chunk: 34/ RREP será rechazado, situación que será informada a el(la) estudiante y al(los) docente(s) vía email, por el(la) director(a) de carrera. El Servicio de Salud Estudiantil o Dirección de Carrera, ti ...
Metadata: {'chunk_id': 'ReglamentodeRegimendeEstudios__2023__35_0', 'doc_id': 'doc_002', 'filename': 'data/raw/ReglamentodeRegimendeEstudios__2023_.pdf', 'page': 35.0, 'source_type': 'pdf', 'title': 'ReglamentodeRegimendeEstudios  2023 ', 'url': 'https://www.ufro.cl', 'vigencia': None}
--------------------------------------------------------
Chunk: 12/ RREP especializada, en concordancia con lo expresado en el perﬁl deﬁnido para cada carrera. El máximo de requisitos que se podrá incluir en una asignatura del Plan de Estudios es de dos, exceptuan ...
Metadata: {'chunk_id': 'ReglamentodeRegimendeEstudios__2023__13_0', 'doc_id': 'doc_002', 'filename': 'data/raw/ReglamentodeRegimendeEstudios__